In [2]:
import requests
import os
import pandas as pd
from xml.dom import minidom
import xml.etree.ElementTree as et
import numpy as np

df_rows = []
df_columns = ["itemid", "headline","text", "bip:topics", "dc.date.published", "XML filename"]
files_path = (r'C:\Users\satya\Downloads\Goutham\mlbd\New folder\New folder\New folder')
files = []
column_bip_topics = []
for r, d, f in os.walk(files_path):
    for file in f:
        if '.xml' in file:
            files.append(os.path.join(r,file))
#-----------------------Step 3--------------------------------------------------------------
def GetDataFrame(files): ##Function to extract DataFrame 
    for name in files:
        address = name.split('\\')
        xml_filename = address[-1] ##GET FILE NAME
        xml_bip_topics = ''
        xml_text = ''
        dom = et.parse(name)
        root = dom.getroot()
        xml_itemId = root.attrib.get("itemid") ##GET ITEM ID
        xml_heading = root.find("headline").text ##GET HEADLINE
        for child in root:
                if(child.tag == 'text'):
                    for textnode in child:
                        if(textnode.tag == 'p'):
                            #print(dir(textnode))
                            xml_text = xml_text + " " + textnode.text##GET TEXT
                for j in child.iter('metadata'):
                    for bip in j.iterfind(".//codes[@class='bip:topics:1.0']"):
                            for bipnodes in bip.iter("code"):
                                xml_bip_topics = bipnodes.attrib.get("code") + "," + xml_bip_topics ##GET BIP_CODES
                    for date in j.iterfind(".//dc[@element='dc.date.published']"):
                                   xml_date=date.get('value') ##GET DATE
        column_bip_topics.append(xml_bip_topics)
        df_rows.append({"headline" : xml_heading, "text" : xml_text, "bip:topics" : xml_bip_topics, "dc.date.published" : xml_date,  "itemid" : xml_itemId,  "XML filename" : xml_filename })
    Final_DF = pd.DataFrame(df_rows, columns = df_columns)
    return Final_DF
Final_DF_Values = GetDataFrame(files)
Text_Column = (Final_DF_Values.loc[:,['text']]).values

In [3]:
#-----------------------Step 4--------------------------------------------------------------
def GetAllBipTopics(column_bip_topics):
    all_bips = []
    first_bip = []
    for b in column_bip_topics:
        indiv_bips = str(b).split(",")
        for bip in indiv_bips:
            all_bips.append(bip)
        first_bip.append(indiv_bips[0])
    All_Indivitual_bips = set(all_bips)
    All_Indivitual_bips = list(filter(None,All_Indivitual_bips)) ##All possible bip values
    return (All_Indivitual_bips,first_bip)
All_Indivitual_bips,first_bip = GetAllBipTopics(column_bip_topics)

In [4]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import sys
##-----------------------Step 5--------------------------------------------------------------
def GetFiltedText(Text_Column):
    WL = WordNetLemmatizer()
    PS = PorterStemmer()
    StopWords = set(stopwords.words('english'))
    Filtered_WordList = []
    for text in Text_Column:
        FilterNumber1 = (re.sub('[^A-Za-z ]+', '', str(text))).lower() ## To lower and Selecting only alphabets
        Tokens = word_tokenize(FilterNumber1)
        FilterNumber2 = []
        Filtered_Sentence = ""
        for word in Tokens:
            if word not in StopWords:
                FilterNumber2.append(PS.stem(WL.lemmatize(word))) ##Stemming, lemmatizing and removing stop words
        for i in FilterNumber2:
            if(len(i) != 1 and len(i) != 2):
                Filtered_Sentence = Filtered_Sentence + " " + i
        Filtered_WordList.append(Filtered_Sentence)
    return Filtered_WordList
Filtered_WordList = GetFiltedText(Text_Column)
print(len(Filtered_WordList))
print(len(first_bip))

48375
48375


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer as TV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
##-----------------------Step 6--------------------------------------------------------------
def FeatureSelection(Filtered_WordList,first_bip):
    model = TV(max_features = 2000) ##Extract more frequency tokens first
    tfidf = model.fit_transform(Filtered_WordList)
    Feature_Name = model.get_feature_names()
    bestfeatures = SelectKBest(score_func=chi2, k=1500) ##Select features based on labels using SelectKBest and chi2
    fit = bestfeatures.fit(tfidf.toarray(),first_bip)
    final_features = fit.transform(tfidf.toarray())
    return final_features
final_features = FeatureSelection(Filtered_WordList, first_bip)

In [20]:
##-----------------------Step 7 & 9--------------------------------------------------------------
from sklearn import model_selection
def CrossValidation(model):
    score = model_selection.cross_val_score(model,final_features, first_bip, scoring='f1_weighted',cv=3)
    return score

Step 7: 
Since we have a large data set, using k-fold cross validation with a small k value(3 to 5) is the best way to divide train and test sets. Doing normal train set-test set split on imbalanced class distribution data set may lead to uneven distribution where a complete n-labbeled cases may fall in test set. And since our data set size is high, using a large number for 'k' will be computationally expensive. Hence small k value. We can mitigate overfitting as well since every data row is used in train set more than once.

In [24]:
##-----------------------Step 8--------------------------------------------------------------
def SGDClasifier():
    from sklearn.linear_model import SGDClassifier
    from sklearn.metrics import accuracy_score
    model = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
    score = CrossValidation(model)
    return score
Score = SGDClasifier()
print(Score.mean())

0.7547668932598809


Step 9: 
The best evaluater to use when we have imbalanced class distribution is F1-score. Even though Accuracy is most widely used, this metric treats all classes equally, where as f1 score keeps a balance between recall and precision and gives a better measure of incorrectly classified inputs. Similarly AUC is the complete area under ROC curse and averages all thesholds, where as F1 score points to particular theshold which is more accurate for imbalanced data.

In [22]:
##-----------------------Step 10--------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import f1_score
model = rf(n_estimators = 20)
score = CrossValidation(model)
print(score.mean())

0.7526137585060657


In [25]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
score = CrossValidation(model)
print(score.mean())

0.6863940652848731


In [26]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
score = CrossValidation(model)
print(score.mean())

0.8050344061999762


In [27]:
from sklearn.neural_network import MLPClassifier as mlp
model = mlp(hidden_layer_sizes = (100,300))
score = CrossValidation(model)
print(score.mean())

0.8030854947208944


In [28]:
from sklearn import svm
model = svm.LinearSVC()
score = CrossValidation(model)
print(score.mean())

0.8180552207189958


After tuning all the mentioned classifiers and looking at the respective F1-scores, Linear Support Vector Classifier gave the best results, hence SVM is the best classifier for this Text dataset. However, given high computational power, MLP classifier(Neural Network) can be trained by tuning the hyperparameters(number of hidden layers) for much better results. Since Linear regression is best suited for continuos output, logistic regression is a more suitable for classification problems. Random forest considers multiple bootstrapped sets and trains in an iteration instead of training full dataset at once like decision tree. Hence, Random forest gave better results than decision tree.